In [ ]:
import csv
import os
from datetime import datetime

def add_expense(expenses):
    """Add a new expense to the list"""
    print("\n--- Add New Expense ---")
    date = input("Enter date (YYYY-MM-DD): ")
    category = input("Enter category (Food/Travel/etc): ").capitalize()
    amount = float(input("Enter amount: "))
    description = input("Enter description: ")

    expense = {
        'date': date,
        'category': category,
        'amount': amount,
        'description': description
    }
    expenses.append(expense)
    print("Expense added successfully!")
    return expense  # Return added expense for budget updates

def view_expenses(expenses):
    """Display all stored expenses"""
    print("\n--- All Expenses ---")
    if not expenses:
        print("No expenses found.")
        return

    for i, expense in enumerate(expenses, 1):
        print(f"{i}. Date: {expense['date']} | "
              f"Category: {expense['category']} | "
              f"Amount: ${expense['amount']:.2f} | "
              f"Description: {expense['description']}")

def set_budget():
    """Set monthly budget with month/year context"""
    print("\n--- Set Monthly Budget ---")
    month_year = input("Enter month and year (YYYY-MM): ")
    amount = float(input("Enter budget amount: "))
    return {
        'month_year': month_year,
        'budget': amount,
        'remaining': amount  # Initialize remaining = budget
    }

def save_budget(budget_data, filename="budget.csv"):
    """Save budget data to CSV file, replacing existing entries for the same month/year"""
    try:
        rows = []
        file_exists = os.path.exists(filename)
        
        # Read existing data if file exists
        if file_exists:
            with open(filename, 'r', newline='') as file:
                reader = csv.reader(file)
                header = next(reader)  # Save header
                
                # Collect rows that DON'T match the current month/year
                for row in reader:
                    if row[0] != budget_data['month_year']:
                        rows.append(row)
        
        # Write all data back to file
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file)
            
            # Write header
            writer.writerow(['month_year', 'budget', 'remaining'])
            
            # Write existing rows (excluding same month/year)
            writer.writerows(rows)
            
            # Write new budget data
            writer.writerow([
                budget_data['month_year'],
                budget_data['budget'],
                budget_data['remaining']
            ])
            
        print(f"Budget data saved to {filename}")
        return True
    except Exception as e:
        print(f"Error saving budget: {e}")
        return False

def load_budget(filename="budget.csv"):
    """Load budget data from CSV, creating file with headers if needed"""
    try:
        # Create file with headers if it doesn't exist
        if not os.path.exists(filename):
            with open(filename, 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['month_year', 'budget', 'remaining'])
            print(f"Created new budget file: {filename}")
            return None
        
        # Load existing budget data
        with open(filename, 'r') as file:
            reader = csv.reader(file)
            header = next(reader)  # Skip header
            row = next(reader, None)
            
            if row:
                return {
                    'month_year': row[0],
                    'budget': float(row[1]),
                    'remaining': float(row[2])
                }
        return None
    except Exception as e:
        print(f"Error loading budget: {e}")
        return None

def update_budget(expenses, budget_data):
    """Calculate remaining budget based on expenses"""
    if not budget_data:
        return None
        
    # Calculate expenses for budget month
    monthly_expenses = sum(
        float(exp['amount']) for exp in expenses
        if exp['date'].startswith(budget_data['month_year'])
    )
    
    # Update remaining amount
    budget_data['remaining'] = budget_data['budget'] - monthly_expenses
    return budget_data

def track_budget(budget_data):
    """Display current budget status"""
    if not budget_data:
        print("\nNo budget set for this month!")
        return

    print(f"\n--- Budget Tracking ({budget_data['month_year']}) ---")
    print(f"Monthly Budget: ${budget_data['budget']:.2f}")
    print(f"Current Remaining: ${budget_data['remaining']:.2f}")

    if budget_data['remaining'] < 0:
        print(f"⚠️ Deficit: ${abs(budget_data['remaining']):.2f} over budget!")

def save_expenses(expenses, budget_data, filename="expenses.csv"):
    """Save expenses to CSV file"""
    try:
        with open(filename, 'w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=['date', 'category', 'amount', 'description'])
            writer.writeheader()
            writer.writerows(expenses)
        print(f"\nExpenses saved to {filename}")
        
        # Save budget data as well
        save_budget(budget_data)
        return True
    except Exception as e:
        print(f"Error saving expenses: {e}")
        return False

def load_expenses(filename="expenses.csv"):
    """Load expenses from CSV file"""
    expenses = []
    try:
        if os.path.exists(filename):
            with open(filename, 'r') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    # Convert amount to float
                    row['amount'] = float(row['amount'])
                    expenses.append(row)
           # print(f"Loaded expenses from {filename}")
        return expenses
    except Exception as e:
        print(f"Error loading expenses: {e}")
        return []

def main():
    """Main program function"""
    expenses = load_expenses()
    budget_data = load_budget()
    print(budget_data)
    
    # Initialize budget if not loaded
    if not budget_data:
        print("Please set your monthly budget")
        budget_data = set_budget()
        save_budget(budget_data)
    else:
        # Update budget with loaded expenses
        budget_data = update_budget(expenses, budget_data)

    while True:
        print("\n===== Personal Expense Tracker =====")
        print("1. Add Expense")
        print("2. View Expenses")
        print("3. Track Budget")
        print("4. Save Expenses")
        print("5. Exit")

        choice = input("Enter your choice (1-5): ")

        if choice == '1':
            # Add expense and update budget
            new_expense = add_expense(expenses)
            budget_data = update_budget([new_expense], budget_data)
        elif choice == '2':
            view_expenses(expenses)
        elif choice == '3':
            # Update and track budget
            budget_data = update_budget(expenses, budget_data)
            track_budget(budget_data)
        elif choice == '4':
            save_expenses(expenses, budget_data)
        elif choice == '5':
            if input("Save before exiting? (y/n): ").lower() == 'y':
                save_expenses(expenses, budget_data)
            print("Goodbye!")
            break
        else:
            print("Invalid choice. Please enter 1-5.")

if __name__ == "__main__":
    main()

{'month_year': '2025-06', 'budget': 3000.0, 'remaining': -1000.0}

===== Personal Expense Tracker =====
1. Add Expense
2. View Expenses
3. Track Budget
4. Save Expenses
5. Exit
